# EDA (Exploratory Data Analysis)

Vamos a realizar un análisis exploratorio de los datos para entender mejor el problema y los datos que tenemos disponibles.

In [22]:
from _utils import *
import pandas as pd

## **`Cargar los datos`**
Primero cargamos los datos y vemos sus dimensiones.

In [23]:
# Cargamos la informacion de las monedas actualmente
df_coins: pd.DataFrame = pd.read_json('../../datasets/json/coins_clean.json', orient='records')
# Cargamos la informacion de cada moneda durante cada halving
df_halving: pd.DataFrame = pd.read_json('./metadata/halving.json', orient='records')

In [24]:
df_coins.shape

(7862, 29)

In [25]:
df_halving.shape

(2126, 4)

## **`Explorar los datos`**
Luego, exploramos los datos para entender mejor las características de los datos.

In [26]:
df_coins.head(3)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,69435.00,1.368316e+12,1,1.457921e+12,3.383568e+10,71931.0,...,-5.84888,2024-03-14T07:10:36.635Z,67.810000,102283.29855,2013-07-06T00:00:00.000Z,2024-06-08T03:23:56.040Z,Proof of Memes Ecosystem,proof-of-memes-ecosystem,"[binance, bybit_spot, nami_exchange, okex, gat...","[thorswap, cube, tanx]"
1,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,69422.00,1.368146e+12,1,1.457740e+12,3.369057e+10,71931.0,...,-5.90683,2024-03-14T07:10:36.635Z,67.810000,102220.28000,2013-07-06T00:00:00.000Z,2024-06-08T03:12:35.413Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, okex, gat...","[thorswap, cube, tanx]"
2,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,3686.14,4.428759e+11,2,4.428759e+11,1.614996e+10,3844.4,...,-24.44817,2021-11-10T14:24:19.604Z,0.432979,851122.83528,2015-10-20T00:00:00.000Z,2024-06-08T03:12:31.225Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, okex, gda...","[curve_ethereum, thorswap, cube, openocean_fin..."


`df_coins` es un DataFrame que contiene la información de las monedas.

In [27]:
df_halving.head(3)

,2012-02-12,2016-02-07,2020-03-05,2024-04-20
0,"{'id': '0016aff8-c747-4a03-8e66-e17abaa3e346',...","{'id': '0016aff8-c747-4a03-8e66-e17abaa3e346',...","{'id': '0016aff8-c747-4a03-8e66-e17abaa3e346',...","{'id': '0016aff8-c747-4a03-8e66-e17abaa3e346',..."
1,"{'id': '001adcf3-e807-41b4-a865-92bfce293a55',...","{'id': '001adcf3-e807-41b4-a865-92bfce293a55',...","{'id': '001adcf3-e807-41b4-a865-92bfce293a55',...","{'id': '001adcf3-e807-41b4-a865-92bfce293a55',..."
2,"{'id': '0027fc83-a360-4197-8bb1-b2efaa767558',...","{'id': '0027fc83-a360-4197-8bb1-b2efaa767558',...","{'id': '0027fc83-a360-4197-8bb1-b2efaa767558',...","{'id': '0027fc83-a360-4197-8bb1-b2efaa767558',..."


`df_halving` es un DataFrame que contiene la información del mercado durante cada halving *(2012, 2016, 2020, 2024)* 

## **`Preprocesamiento de los datos`**
Finalmente, realizamos un preprocesamiento de los datos para poder utilizarlos en el entrenamiento de un modelo de machine learning.


Primero vamos a convertir cada halving en un dataset de serie de tiempo para manejar mas facilmente la data

In [28]:
def proccess_coin_in_time(coin: dict) -> list[dict]:
    # new_coin: dict = coin.copy()
    new_df: list = []
    # del new_coin["contractAddresses"]
    # del new_coin["halving"]
    # del new_coin["allTimeHighData"]
    # del new_coin["cycleLowData"]
    timestamp = coin["halving"]
    if type(timestamp) == str:
        # new_coin["datetime"] = None
        # new_coin["open"] = None
        # new_coin["high"] = None
        # new_coin["low"] = None
        # new_coin["close"] = None
        # new_coin["volume"] = None
        # new_df.append(new_coin)
        return new_df
    else:
        for time in timestamp:
            new_coin = coin.copy()
            del new_coin["contractAddresses"]
            del new_coin["halving"]
            del new_coin["allTimeHighData"]
            del new_coin["cycleLowData"]
            new_coin["datetime"] = datetime.fromtimestamp(time["timestamp"])
            new_coin["open"] = time["open"]
            new_coin["high"] = time["high"]
            new_coin["low"] = time["low"]
            new_coin["close"] = time["close"]
            new_coin["volume"] = time["volume"]
            new_df.append(new_coin)
    return new_df

In [29]:
def generate_halving_df(halving: pd.Series) -> pd.DataFrame:
    df = pd.DataFrame()
    for coin in halving:
        new_df= proccess_coin_in_time(coin)
        df = pd.concat([df, pd.DataFrame(new_df)])
    return df

In [30]:
def generate_datasets(dataset: pd.DataFrame) -> None:
    # Generamos los datasets de los halving de menor capitalizacion
    for i, halving in enumerate(dataset):
        df_halving: pd.DataFrame = generate_halving_df(dataset[halving])
        df_halving.to_json(f"./datasets/{i+1}_halving.json", orient="records")

In [31]:
generate_datasets(df_halving)

Eliminamos variables para evitar problemas de memoria

In [32]:
del df_halving

## **`Visualización de los datos`**
Visualizamos los datos para entender mejor las características de los datos y cómo se relacionan entre sí.

In [33]:
import matplotlib.pyplot as plt

In [34]:
df_halving_1 = pd.read_json('./datasets/1_halving.json', orient='records')
df_halving_2 = pd.read_json('./datasets/2_halving.json', orient='records')
df_halving_3 = pd.read_json('./datasets/3_halving.json', orient='records')
df_halving_4 = pd.read_json('./datasets/4_halving.json', orient='records')

In [35]:
df_halving_1.shape, df_halving_2.shape, df_halving_3.shape, df_halving_4.shape

((326, 11), (6223, 11), (74870, 11), (87497, 11))

Visualizaremos el rendimiento de cada moneda durante el halving

In [36]:
df_halving_1.set_index("datetime", inplace=True)
df_halving_2.set_index("datetime", inplace=True)
df_halving_3.set_index("datetime", inplace=True)
df_halving_4.set_index("datetime", inplace=True)

In [40]:
def plot_by_coins(coin_1: str, coin_2: str) -> None:
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))

    # Ordena y grafica para coin_1 en High Halving 1
    df_high_halving_1_sorted = df_halving_1[df_halving_1["name"] == coin_1].sort_index()
    df_low_halving_1_sorted = df_halving_1[df_halving_1["name"] == coin_2].sort_index()
    axs[0, 0].plot(df_high_halving_1_sorted["close"], label="High Halving 1")
    axs[0, 0].plot(df_low_halving_1_sorted["close"], label="High Halving 1")
    axs[0, 0].set_title(f"{coin_1} vs {coin_2}")
    axs[0, 0].legend()

    # Ordena y grafica para coin_1 en High Halving 2 y coin_2 en Low Halving 2
    df_high_halving_2_sorted = df_halving_2[df_halving_2["name"] == coin_1].sort_index()
    df_low_halving_2_sorted = df_halving_2[df_halving_2["name"] == coin_2].sort_index()
    axs[0, 1].plot(df_high_halving_2_sorted["close"], label="High Halving 2")
    axs[0, 1].plot(df_low_halving_2_sorted["close"], label="Low Halving 3")
    axs[0, 1].set_title(f"{coin_1} vs {coin_2}")
    axs[0, 1].legend()

    # Ordena y grafica para coin_1 en High Halving 3 y coin_2 en Low Halving 3
    df_high_halving_3_sorted = df_halving_3[df_halving_3["name"] == coin_1].sort_index()
    df_low_halving_3_sorted = df_halving_3[df_halving_3["name"] == coin_2].sort_index()
    axs[1, 0].plot(df_high_halving_3_sorted["close"], label="High Halving 3")
    axs[1, 0].plot(df_low_halving_3_sorted["close"], label="Low Halving 4")
    axs[1, 0].set_title(f"{coin_1} vs {coin_2}")
    axs[1, 0].legend()

    # Ordena y grafica para coin_1 y coin_2 en High Halving 4 y Low Halving 4 respectivamente
    # Aquí, df_low_halving_4_sorted ya está ordenado de antes, así que lo reutilizamos
    df_high_halving_4_sorted = df_halving_4[df_halving_4["name"] == coin_1].sort_index()
    df_low_halving_4_sorted = df_halving_4[df_halving_4["name"] == coin_2].sort_index()
    axs[1, 1].plot(df_high_halving_4_sorted["close"], label="High Halving 4")
    axs[1, 1].plot(df_low_halving_4_sorted["close"], label="Low Halving 4")
    axs[1, 1].set_title(f"{coin_1} vs {coin_2}")
    axs[1, 1].legend()

    plt.show()

In [41]:
plot_by_coins("Bitcoin","Parallax")

In [44]:
def plot_by_halving(halving: pd.DataFrame):
    plt.figure(figsize=(10, 15))
    ax = plt.gca()
    ax.set_yscale('log')

    # Encuentra la fecha mínima (inicio) entre todos los grupos
    start_date = halving.index.min()
    # Calcula la fecha objetivo sumando 150 días a la fecha de inicio
    target_date = start_date + pd.DateOffset(days=165)

    for name, group in halving.groupby("name"):
        sorted_group = group.sort_index()
        sorted_group.plot.line(y="close", label=name, ax=ax)

    # Trazar una línea vertical en la fecha objetivo
    ax.axvline(x=target_date, color='red', linestyle='--', label='150 days from start')

    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.show()

# Asumiendo que df_high_halving_2 está definido previamente y correctamente indexado por fecha
plot_by_halving(df_halving_2)